In [ ]:
import pandas as pd
import numpy as np
import json, os, math
from collections import defaultdict as ddict

In [ ]:
# adding grades for another project just involves changing these and re-running

# file exported from Canvas
file_a = "canvas-v22.csv"

# new file we'll create to be imported to Canvas
file_b = "canvas-v23.csv"

# will update this project and all before it
last_proj = 9

# this directory contains p1.json, p2.json, etc
grades_dir = ("/Users", "trh", "g", "caraza-harter-com", "tools", "grades")

# how many late days can they use before they get ZERO?
late_allowed = 9

In [ ]:
# c is the canvas DataFrame
c = pd.read_csv(file_a)
c.head()

In [ ]:
def proj_col(num):
    pcol = None
    for col in list(c):
        if col.startswith("P"+str(num)+" "):
            pcol = col
    assert(pcol != None)
    return pcol
proj_col(last_proj)

In [ ]:
idx = c["SIS Login ID"].str.split("@", expand=True)[0].str.lower()
idx = idx.rename("")
idx[0] = "points" # FIX this after tweaks
c.set_index(idx, inplace=True)
c.head()

In [ ]:
# we'll merge all project grades here, ordered by project number
rows = []

# loop over project JSON files, updating Canvas
for i in range(1, last_proj+1):
    path = grades_dir + ("p%d.json"%i,)
    path = os.path.join(*path)
    with open(path) as f:
        rows += json.load(f)

In [ ]:
c

In [ ]:
total_late = ddict(int)

# update Canvas grades, cell-by-cell
for row in rows:
    net_id = row["net_id"]
    if not net_id in c.index:
        continue # they dropped?f
    
    pcol = proj_col(row["project"][1:])
    possible = float(c.loc["points", pcol])
    score = (row["score"] / 100) * possible
    late = row["late_days"]
    
    if late > 0:
        total_late[net_id] += late
        if total_late[net_id] > late_allowed:
            warn = "WARNING: not counting '%s' grade of %d for '%s' due to lateness"
            print(warn % (pcol, score, net_id))
            score = 0

    # update Canvas if new grade is better
    prev = float(c.loc[net_id, pcol])
    if math.isnan(prev) or score >= prev:
        # increase their canvas score
        c.loc[net_id, pcol] = score
    else:
        warn = "WARNING: not reducing '%s' grade for '%s' from %.1f to %.1f"
        print(warn % (pcol, net_id, prev, score))

# apply zeros for cheating

In [ ]:
def set_proj(df, net_id, proj, score):
    for col in df.columns:
        if col.split(" ")[0].lower() == proj.lower():
            break
    else:
        raise Exception("could not find {} {}".format(net_id, proj))
    assert net_id in df.index
    df.loc[net_id, col] = score
        
with open("cheating.json") as f:
    cheating = json.load(f)
for proj, students in cheating.items():
    for net_id, score in students.items():
        print("Set score to {} on {} for {}".format(score, proj, net_id))
        set_proj(c, net_id, proj, score)

In [ ]:
# create new Canvas CSV that we can import back in
c.to_csv(file_b, index=False)